In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time

from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines

In [12]:
features = pd.read_csv('fire_features.csv')


"""
Check where there are zeros so we can remove those rows
print(np.where(features['TLML_12Z']==0))
print(np.where(features['QLML_12Z']==0))
print(np.where(features['SPEEDLML_12Z']==0))
print(np.where(features['PS_12Z']==0))
print(np.where(features['T_12Z_500mb']==0))
print(np.where(features['QV_12Z_700mb']==0))
print(np.where(features['TLML_12Z']==0))"""

features = features.drop('Unnamed: 0', axis=1)
inds = np.where(features['QV_12Z_700mb']==0) #here's where we drop the zero features
features = features.drop(labels=inds[0], axis=0)
features = features.reset_index(drop=True) #reset the indices beecause we dropped some
features

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,TLML_12Z,QLML_12Z,SPEEDLML_12Z,PS_12Z,T_12Z_700mb,T_12Z_500mb,QV_12Z_700mb,biomass_12Z
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,290.748108,0.004608,10.770707,91644.742188,281.218140,261.982361,0.001890,7.627477e-10
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,290.003723,0.003563,8.092171,81498.414062,286.447601,266.682251,0.002318,0.000000e+00
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,281.681793,0.006098,10.386342,82444.875000,281.240356,262.869751,0.007128,0.000000e+00
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,283.622253,0.009279,6.614388,81877.890625,283.966187,263.090515,0.005877,0.000000e+00
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,284.536926,0.006591,12.682461,81796.609375,284.776917,267.068909,0.004644,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8090,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,293.930328,0.003627,7.339819,91107.664062,282.576782,262.727051,0.004921,0.000000e+00
8091,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,279.254852,0.001604,6.979411,90997.539062,276.615021,257.887451,0.001588,1.553367e-09
8092,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,288.125366,0.001511,10.678336,99387.382812,275.902649,258.410187,0.000153,0.000000e+00
8093,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,287.946075,0.002882,5.147175,99128.710938,272.194824,258.612366,0.002691,2.873148e-10


In [13]:
# calculate the hot-dry-windy and related variables
print(len(features['TLML_12Z'].values))
esat = sat_vap_press(features['TLML_12Z'].values)
e = vap_press(features['QLML_12Z'].values, features['PS_12Z'].values)
hdw = hot_dry_windy(e, esat, features['SPEEDLML_12Z'].values)
print(len(esat))

#append them to the dataframe
df_hdw = pd.DataFrame({'ESATLML_12Z':esat, 'ELML_12Z':e, 'HDWLML':hdw})
print(df_hdw)
features = pd.concat([features, df_hdw], axis=1)
features

8095
8095
      ESATLML_12Z  ELML_12Z      HDWLML
0       20.131931  2.638775  188.413662
1       19.205433  1.812535  140.746304
2       11.125224  3.146602   82.868697
3       12.676732  4.770291   52.296270
4       13.471937  3.375874  128.042935
...           ...       ...         ...
8090    24.548924  2.063010  165.042546
8091     9.421946  0.909643   59.410857
8092    17.031751  0.935601  171.880108
8093    16.836034  1.782489   77.483227
8094    17.031751  0.935601  171.880108

[8095 rows x 3 columns]


,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,TLML_12Z,QLML_12Z,SPEEDLML_12Z,PS_12Z,T_12Z_700mb,T_12Z_500mb,QV_12Z_700mb,biomass_12Z,ESATLML_12Z,ELML_12Z,HDWLML
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,290.748108,0.004608,10.770707,91644.742188,281.218140,261.982361,0.001890,7.627477e-10,20.131931,2.638775,188.413662
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,290.003723,0.003563,8.092171,81498.414062,286.447601,266.682251,0.002318,0.000000e+00,19.205433,1.812535,140.746304
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,281.681793,0.006098,10.386342,82444.875000,281.240356,262.869751,0.007128,0.000000e+00,11.125224,3.146602,82.868697
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,283.622253,0.009279,6.614388,81877.890625,283.966187,263.090515,0.005877,0.000000e+00,12.676732,4.770291,52.296270
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,284.536926,0.006591,12.682461,81796.609375,284.776917,267.068909,0.004644,0.000000e+00,13.471937,3.375874,128.042935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8090,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,293.930328,0.003627,7.339819,91107.664062,282.576782,262.727051,0.004921,0.000000e+00,24.548924,2.063010,165.042546
8091,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,279.254852,0.001604,6.979411,90997.539062,276.615021,257.887451,0.001588,1.553367e-09,9.421946,0.909643,59.410857
8092,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,288.125366,0.001511,10.678336,99387.382812,275.902649,258.410187,0.000153,0.000000e+00,17.031751,0.935601,171.880108
8093,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,287.946075,0.002882,5.147175,99128.710938,272.194824,258.612366,0.002691,2.873148e-10,16.836034,1.782489,77.483227


In [14]:
# calculate the Haines and related variables
import metpy.calc as mc
from metpy.units import units

#vapor pressure at 700mb
e_hPa_700 = vap_press(features['QV_12Z_700mb'].values, 700*np.ones(features['QV_12Z_700mb'].values.shape)) # hPa
e_hPa_700 = units.Quantity(e_hPa_700, "hPa")

#dewpoint at 700mb
td_700 = mc.dewpoint(e_hPa_700)
td_700 = np.array(td_700) # degrees C
print(td_700.shape)

haines_index = haines(features['T_12Z_700mb'].values, features['T_12Z_500mb'], td_700)

df_haines = pd.DataFrame({'Td_12Z_700mb':td_700, 'E_700mb':e_hPa_700, 'HAINES': haines_index})

features = pd.concat([features, df_haines], axis=1)
features

(8095,)


,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,TLML_12Z,QLML_12Z,SPEEDLML_12Z,PS_12Z,T_12Z_700mb,T_12Z_500mb,QV_12Z_700mb,biomass_12Z,ESATLML_12Z,ELML_12Z,HDWLML,Td_12Z_700mb,E_700mb,HAINES
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,290.748108,0.004608,10.770707,91644.742188,281.218140,261.982361,0.001890,7.627477e-10,20.131931,2.638775,188.413662,-66.278872,0.008245,5.0
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,290.003723,0.003563,8.092171,81498.414062,286.447601,266.682251,0.002318,0.000000e+00,19.205433,1.812535,140.746304,-64.771856,0.010118,5.0
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,281.681793,0.006098,10.386342,82444.875000,281.240356,262.869751,0.007128,0.000000e+00,11.125224,3.146602,82.868697,-55.985997,0.031258,5.0
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,283.622253,0.009279,6.614388,81877.890625,283.966187,263.090515,0.005877,0.000000e+00,12.676732,4.770291,52.296270,-57.560401,0.025739,5.0
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,284.536926,0.006591,12.682461,81796.609375,284.776917,267.068909,0.004644,0.000000e+00,13.471937,3.375874,128.042935,-59.441913,0.020317,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8090,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,293.930328,0.003627,7.339819,91107.664062,282.576782,262.727051,0.004921,0.000000e+00,24.548924,2.063010,165.042546,-58.983160,0.021533,5.0
8091,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,279.254852,0.001604,6.979411,90997.539062,276.615021,257.887451,0.001588,1.553367e-09,9.421946,0.909643,59.410857,-67.541377,0.006927,5.0
8092,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,288.125366,0.001511,10.678336,99387.382812,275.902649,258.410187,0.000153,0.000000e+00,17.031751,0.935601,171.880108,-82.935717,0.000664,5.0
8093,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,287.946075,0.002882,5.147175,99128.710938,272.194824,258.612366,0.002691,2.873148e-10,16.836034,1.782489,77.483227,-63.654890,0.011750,4.0


In [15]:
features.to_csv('fire_features_with_derived.csv', index =False)